# Cost estimator

## Data cleanup

### Importing libraries

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
crops = [
    "Apple",
    "Banana",
    "Blackgram",
    "Chickpea",
    "Coconut",
    "Coffee",
    "Cotton",
    "Grapes",
    "Jute",
    "Kidneybeans",
    "Lentil",
    "Maize",
    "Mango",
    "Mothbeans",
    "Mungbeans",
    "Orange",
    "Papaya",
    "Pigeonpeas",
    "Pomegranate",
    "Rice",
]
datasets = {}
for crop in crops:
    datasets[crop] = pd.read_csv(f"datasets/raw/crops/2023/{crop}.csv")
print(datasets)

{'Apple':              state district      market commodity   variety arrival_date  \
0            Bihar   Araria  Forbesganj     Apple  American   22/01/2023   
1            Bihar   Araria  Forbesganj     Apple  American   23/01/2023   
2            Bihar   Araria  Forbesganj     Apple  American   24/01/2023   
3            Bihar   Araria  Forbesganj     Apple  American   25/01/2023   
4            Bihar   Araria  Forbesganj     Apple  American   26/01/2023   
...            ...      ...         ...       ...       ...          ...   
70512  West Bengal  Kolkata      Mechua     Apple     Other   06/05/2023   
70513  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70514  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70515  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   
70516  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   

       min_price  max_price  modal_price update_date  
0         6000.0     9

In [23]:
location_data = pd.read_csv("datasets/raw/district_coords.csv")
def get_coords(state, district):
    # global count
    i = location_data[location_data["District"]==district]
    i = i[i["State"]==state.upper()]
    if i.shape[0] != 1:
        # with open("not_found.csv", "a") as f:
        #     f.write(f"{state},{district},{i.shape[0]}\n")
        # count += 1
        return (0,0)
    return (float(i["Latitude"].iloc[0]), float(i["Longitude"].iloc[0]))

def convert_date(date):
    return datetime.strptime(date, "%d/%m/%Y").toordinal()

In [4]:
print(get_coords(datasets["Apple"]["state"][4], datasets["Apple"]["district"][4]))

(26.137794, 87.464469)


In [5]:
print(datasets["Apple"])

             state district      market commodity   variety arrival_date  \
0            Bihar   Araria  Forbesganj     Apple  American   22/01/2023   
1            Bihar   Araria  Forbesganj     Apple  American   23/01/2023   
2            Bihar   Araria  Forbesganj     Apple  American   24/01/2023   
3            Bihar   Araria  Forbesganj     Apple  American   25/01/2023   
4            Bihar   Araria  Forbesganj     Apple  American   26/01/2023   
...            ...      ...         ...       ...       ...          ...   
70512  West Bengal  Kolkata      Mechua     Apple     Other   06/05/2023   
70513  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70514  West Bengal  Kolkata      Mechua     Apple     Other   15/06/2023   
70515  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   
70516  West Bengal  Kolkata      Mechua     Apple     Other   30/10/2023   

       min_price  max_price  modal_price update_date  
0         6000.0     9000.0     

Add latitude and longitude to all the datapoints

In [29]:
# count = 0
new_datasets = {}
for crop in ["Maize"]:
    dataset = datasets[crop]
    coords = [get_coords(e["state"], e["district"]) for _,e in dataset.iterrows()]
    lats = [coord[0] for coord in coords]
    longs = [coord[1] for coord in coords]
    new_datasets[crop] = dataset.assign(latitude=lats).assign(longitude=longs).drop(["state","district","market","variety", "update_date"],axis=1)
    new_datasets[crop].to_csv(f"datasets/processed/crops/{crop}.csv", sep=",", index=False)
    new_datasets[crop]["arrival_date"] = new_datasets[crop]["arrival_date"].apply(convert_date)
print(new_datasets)

{'Maize':       commodity  arrival_date  min_price  max_price  modal_price  latitude  \
0         Maize        738631     1980.0     2100.0       2000.0       0.0   
1         Maize        738634     1980.0     2100.0       2000.0       0.0   
2         Maize        738635     1980.0     2050.0       2000.0       0.0   
3         Maize        738636     1980.0     2050.0       2000.0       0.0   
4         Maize        738638     1980.0     2050.0       2000.0       0.0   
...         ...           ...        ...        ...          ...       ...   
55543     Maize        738754     1700.0     1850.0       1740.0       0.0   
55544     Maize        738758     1700.0     1800.0       1750.0       0.0   
55545     Maize        738763     1750.0     1770.0       1760.0       0.0   
55546     Maize        738775     1800.0     1850.0       1810.0       0.0   
55547     Maize        738776     1800.0     1850.0       1810.0       0.0   

       longitude  
0            0.0  
1            0.

In [30]:
new_datasets["Maize"].dropna()
new_datasets["Maize"].to_csv("datasets/processed/crops/Maize2.csv")

# RNN Model

## Preparing data for RNN

In [1]:
from keras.layers import Dense, LSTM, Input
from keras.models import Sequential, Model
from keras.optimizers import Adam
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
from datetime import datetime
import numpy as np
import pandas as pd

np.random.seed(42)

2023-12-02 20:20:53.509609: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-02 20:20:53.510986: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-02 20:20:53.532015: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 20:20:53.532039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 20:20:53.532651: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
df = pd.read_csv(f"/tmp/tp.csv")
print(df[['min_price', 'max_price', 'modal_price']])

      min_price  max_price  modal_price
0        7500.0     9055.0         8500
1        4000.0     8500.0         7500
2        7000.0     9500.0         8500
3        4000.0     5000.0         4500
4        4000.0     8775.0         6000
...         ...        ...          ...
9582     9550.0     9725.0         9650
9583     9575.0     9725.0         9650
9584     9570.0     9735.0         9650
9585     9565.0     9735.0         9660
9586     9600.0     9725.0         9660

[9587 rows x 3 columns]


In [3]:
print(df)

                     commodity  arrival_date  min_price  max_price  \
0     Black Gram Dal (Urd Dal)        738788     7500.0     9055.0   
1     Black Gram Dal (Urd Dal)        738791     4000.0     8500.0   
2     Black Gram Dal (Urd Dal)        738793     7000.0     9500.0   
3     Black Gram Dal (Urd Dal)        738796     4000.0     5000.0   
4     Black Gram Dal (Urd Dal)        738797     4000.0     8775.0   
...                        ...           ...        ...        ...   
9582  Black Gram Dal (Urd Dal)        738823     9550.0     9725.0   
9583  Black Gram Dal (Urd Dal)        738824     9575.0     9725.0   
9584  Black Gram Dal (Urd Dal)        738825     9570.0     9735.0   
9585  Black Gram Dal (Urd Dal)        738826     9565.0     9735.0   
9586  Black Gram Dal (Urd Dal)        738827     9600.0     9725.0   

      modal_price   latitude  longitude  
0            8500  23.845103  72.111466  
1            7500  23.845103  72.111466  
2            8500  23.845103  72.

In [6]:
# Split data into train and cross validation
split_date = np.percentile(df["arrival_date"], 80)
print(datetime.fromordinal(np.min(df["arrival_date"])))
print(datetime.fromordinal(int(split_date)))
print(datetime.fromordinal(np.max(df["arrival_date"])))
mask = df["arrival_date"] < split_date

train_df = df[mask].copy()
test_df = df[~mask].copy()

2023-01-01 00:00:00
2023-08-31 00:00:00
2023-11-05 00:00:00


In [7]:
scaler = MaxAbsScaler()
train_df[['min_price', 'max_price', 'modal_price']] = scaler.fit_transform(train_df[['min_price', 'max_price', 'modal_price']])
test_df[['min_price', 'max_price', 'modal_price']] = scaler.transform(test_df[['min_price', 'max_price', 'modal_price']])

In [8]:
sequence_length = 5  # Need to tune this hyper parameter
batch_size = 16
X_train, y_train = [], []
for i in range(len(train_df) - sequence_length):
    X_train.append(train_df.iloc[i:i+sequence_length][['arrival_date', 'latitude', 'longitude']].values)
    y_train.append(train_df.iloc[i+sequence_length][['min_price', 'max_price', 'modal_price']].values)
# X_train = np.asarray(X_train, dtype=[('uint64', 'float32',  'float32')])
X_train = np.asarray(X_train)
y_train = np.asarray(y_train).astype('float32')


X_test, y_test = [], []
for i in range(len(test_df) - sequence_length):
    X_test.append(test_df.iloc[i:i+sequence_length][['arrival_date', 'latitude', 'longitude']].values)
    y_test.append(test_df.iloc[i+sequence_length][['min_price', 'max_price', 'modal_price']].values)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test).astype('float32')

In [10]:
# Creating sequence for lstm
print(X_train)

[[[7.3852200e+05 1.2977080e+01 7.7596173e+01]
  [7.3852300e+05 1.2977080e+01 7.7596173e+01]
  [7.3852400e+05 1.2977080e+01 7.7596173e+01]
  [7.3852500e+05 1.2977080e+01 7.7596173e+01]
  [7.3852600e+05 1.2977080e+01 7.7596173e+01]]

 [[7.3852300e+05 1.2977080e+01 7.7596173e+01]
  [7.3852400e+05 1.2977080e+01 7.7596173e+01]
  [7.3852500e+05 1.2977080e+01 7.7596173e+01]
  [7.3852600e+05 1.2977080e+01 7.7596173e+01]
  [7.3852700e+05 1.2977080e+01 7.7596173e+01]]

 [[7.3852400e+05 1.2977080e+01 7.7596173e+01]
  [7.3852500e+05 1.2977080e+01 7.7596173e+01]
  [7.3852600e+05 1.2977080e+01 7.7596173e+01]
  [7.3852700e+05 1.2977080e+01 7.7596173e+01]
  [7.3852900e+05 1.2977080e+01 7.7596173e+01]]

 ...

 [[7.3875300e+05 2.5325043e+01 8.2975165e+01]
  [7.3875400e+05 2.5325043e+01 8.2975165e+01]
  [7.3875500e+05 2.5325043e+01 8.2975165e+01]
  [7.3875700e+05 2.5325043e+01 8.2975165e+01]
  [7.3875800e+05 2.5325043e+01 8.2975165e+01]]

 [[7.3875400e+05 2.5325043e+01 8.2975165e+01]
  [7.3875500e+05 2.5

In [11]:
model = Sequential()
model.add(LSTM(3, input_shape=(sequence_length, 3), batch_size=batch_size))
model.add(Dense(32, activation='tanh'))
model.add(Dense(3, activation='linear'))

print(model.summary())
print(y_train.shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (16, 3)                   84        
                                                                 
 dense (Dense)               (16, 32)                  128       
                                                                 
 dense_1 (Dense)             (16, 3)                   99        
                                                                 
Total params: 311 (1.21 KB)
Trainable params: 311 (1.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
(7652, 3)


In [12]:
# Compile the model
# combined_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.compile(optimizer=Adam(), loss='mean_squared_error')

In [13]:
print(X_train.dtype)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

float64
Epoch 1/10
216/216 [==============================] - 1s 3ms/step - loss: 0.0805 - val_loss: 0.0021
Epoch 2/10
216/216 [==============================] - 1s 2ms/step - loss: 0.0057 - val_loss: 0.0024
Epoch 3/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0057 - val_loss: 0.0023
Epoch 4/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss: 0.0024
Epoch 5/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss: 0.0024
Epoch 6/10
216/216 [==============================] - 1s 2ms/step - loss: 0.0058 - val_loss: 0.0021
Epoch 7/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss: 0.0020
Epoch 8/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss: 0.0016
Epoch 9/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss: 0.0021
Epoch 10/10
216/216 [==============================] - 0s 2ms/step - loss: 0.0058 - val_loss

In [14]:
# Evaluate the model
evaluation = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error on Test Set: {evaluation}")

61/61 [==============================] - 0s 821us/step - loss: 0.0094
Mean Absolute Error on Test Set: 0.00935676321387291


In [15]:
print(df[-8:])

                     commodity  arrival_date  min_price  max_price  \
9579  Black Gram Dal (Urd Dal)        738818     9600.0     9725.0   
9580  Black Gram Dal (Urd Dal)        738820     9625.0     9765.0   
9581  Black Gram Dal (Urd Dal)        738821     9565.0     9725.0   
9582  Black Gram Dal (Urd Dal)        738823     9550.0     9725.0   
9583  Black Gram Dal (Urd Dal)        738824     9575.0     9725.0   
9584  Black Gram Dal (Urd Dal)        738825     9570.0     9735.0   
9585  Black Gram Dal (Urd Dal)        738826     9565.0     9735.0   
9586  Black Gram Dal (Urd Dal)        738827     9600.0     9725.0   

      modal_price   latitude  longitude  
9579         9675  25.325043  82.975165  
9580         9680  25.325043  82.975165  
9581         9650  25.325043  82.975165  
9582         9650  25.325043  82.975165  
9583         9650  25.325043  82.975165  
9584         9650  25.325043  82.975165  
9585         9660  25.325043  82.975165  
9586         9660  25.325043  82.

In [17]:
y_pred = model.predict(np.array([
                       [[738823, 25.325, 82.975],
                       [738824, 25.325, 82.975],
                       [738825, 25.325, 82.975],
                       [738826, 25.325, 82.975],
                       [738827, 25.325, 82.975]
                        ]]))
y_pred = scaler.inverse_transform(y_pred)
print(y_pred)



1/1 [==============================] - 0s 207ms/step
[[9218.371 9692.941 9474.003]]


In [18]:
print(X_test[-10:])

(array([[[7.3878800e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879100e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879300e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879600e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879700e+05, 2.3845102e+01, 7.2111465e+01]],

       [[7.3879100e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879300e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879600e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879700e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879900e+05, 2.3845102e+01, 7.2111465e+01]],

       [[7.3879300e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879600e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879700e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3879900e+05, 2.3845102e+01, 7.2111465e+01],
        [7.3880200e+05, 2.3845102e+01, 7.2111465e+01]],

       ...,

       [[7.3881800e+05, 2.5325043e+01, 8.2975166e+01],
        [7.3882000e+05, 2.5325043e+01, 8.2975166e+01],
        [7.3882100e+05, 2.5325043e+01, 8.2975

In [ ]:
#Split the DArray into training:test set(1:9)
train_s = int(len(DArray) * 0.1)
train=  DArray[0:train_s,:]
test = DArray[train_s:len(DArray),:]